<a href="https://colab.research.google.com/github/xiborxu/IST462xibo/blob/main/IST462_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/Crash_Reporting_-_Drivers_Data.csv')
df

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df['Road Name'].nunique()

In [ ]:
df['Cross-Street Name'].nunique()

In [ ]:
df.columns

In [ ]:
columns_drop = ['Report Number', 'Local Case Number', 'Agency Name', 'Off-Road Description', 'Municipality', 'Related Non-Motorist', 'Non-Motorist Substance Abuse', 'Person ID', 'Circumstance','Vehicle ID']
df = df.drop(columns_drop, axis=1)

In [ ]:
df.isnull().sum()